<a href="https://colab.research.google.com/github/Lens3010/Pedestrian-Detector-HOG-SVM/blob/main/Progetto_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emoji
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Li

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, DebertaV2Model
from tqdm import tqdm
from datasets import Dataset
import nltk
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import emoji
import random

In [3]:
gpu_device = 'cuda'

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
train_text_path = "train_text.txt"
train_labels_path = "train_labels.txt"
test_text_path = "test_text.txt"
val_text_path = "val_text.txt"
val_labels_path = "val_labels.txt"
mapping_path = "mapping.txt"

with open(train_text_path, "r", encoding="utf-8") as f:
    texts = [line.strip() for line in f]

with open(train_labels_path, "r", encoding="utf-8") as f:
    labels = [int(line.strip()) for line in f]

with open(test_text_path, "r", encoding="utf-8") as f:
    test_texts = [line.strip() for line in f]

with open(val_text_path, "r", encoding="utf-8") as f:
    val_texts = [line.strip() for line in f]

with open(val_labels_path, "r", encoding="utf-8") as f:
    val_labels = [int(line.strip()) for line in f]



mapping_df = pd.read_csv(mapping_path, sep=" ", header=None, names=["id", "label"])
mapping_dict = dict(zip(mapping_df["id"], mapping_df["label"]))

In [6]:
# Carico il tokenizzatore
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
model = AutoModel.from_pretrained(model_name)

# Token speciali per ironia
special_tokens = {
    "additional_special_tokens": [
        "<ALTERNATE>", "<EXAGGERATION>", "<ELLIPSIS>",
        "<NUMBER>", "<EMOJI>", "<IRONY>", "<EMPHASIS>"
    ]
}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

slang_dictionary = {
    'lol': '<IRONY> lol',
    #'idk': "i don't know",
    #'tbh': 'to be honest',
    #'imo': 'in my opinion',
    'omg': '<EMPHASIS> omg',
    'wtf': '<EMPHASIS> wtf',
    'ikr': '<IRONY> ikr',
    #'smh': '<IRONY> smh',
    #'rn': 'right now',
    #'fr': 'for real',
    #'ngl': 'not gonna lie',
    #'ppl': 'people',
    #'btw': 'by the way',
    #'&': 'and',
    #'w/': 'with',
    'lmao': '<IRONY> lmao',
    'rofl': '<IRONY> rofl'
}

def clean_text(text):
    """Pulizia iniziale del testo"""
    # Rimuovi spazi multipli
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def process_hashtags(text):
    """Processa gli hashtag mantenendoli come token singoli"""
    def split_hashtag(match):
        hashtag = match.group(0)
        words = re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z][a-z]|\d|\W|$)|\d+', hashtag[1:])
        return ' '.join(words).lower()

    return re.sub(r'#\w+', split_hashtag, text)

def process_emojis(text):
    """Converte emoji in token significativi"""
    return emoji.demojize(text)

def detect_irony_patterns(text):
    """Rileva pattern comuni di ironia"""
    # Ripetizione di lettere (es. "sooooo")
    text = re.sub(r'(\w)\1{2,}', r'\1\1 <IRONY>', text)
    # Alternanza maiuscole/minuscole
    text = re.sub(r'\b(?:[a-z][A-Z]|[A-Z][a-z]){4,}\w*\b', lambda m: f"{m.group(0)} <ALTERNATE>", text)
    # Punteggiatura enfatica
    text = re.sub(r'(!|\?){2,}', r'\1 <EMPHASIS>', text)
    return text

def preprocess_text(text):
    # Pulizia iniziale
    text = clean_text(text)

    # Processo emoji prima della tokenizzazione
    text = process_emojis(text)

    # Processo hashtag
    text = process_hashtags(text)

    # Sostituisci URL e menzioni
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'@\w+', '', text)

    # Rileva pattern di ironia
    text = detect_irony_patterns(text)

    # Sostituisci slang
    words = text.split()
    processed_words = []
    for word in words:
        word_lower = word.lower()
        if word_lower in slang_dictionary:
            processed_words.append(slang_dictionary[word_lower])
        else:
            processed_words.append(word)

    text = ' '.join(processed_words)


    return text  # Restituisce SOLO i token

def tokenize_text(text):
    return preprocess_text(text)  # Restituisce lista di token per ogni frase



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

In [7]:
def process_text_batch(texts, labels=None, verbose=True):

    if labels is not None and len(texts) != len(labels):
        raise ValueError("La lunghezza delle liste 'texts' e 'labels' deve essere uguale.")

    processed_texts = []
    processed_labels = []

    # Usa tqdm se verbose è True, altrimenti usa enumerate normale
    iterable = tqdm(texts, desc="Processing texts", total=len(texts)) if verbose else texts

    for idx, text in enumerate(iterable):

        # Verifica se il testo originale è vuoto o None
        if text is None or not text.strip():
            continue

        # Processa il testo
        processed = preprocess_text(text)

        # Verifica se il risultato è vuoto dopo il processamento
        if processed:
            processed_texts.append(processed)

            # Se labels non è None, aggiungi anche l'etichetta
            if labels is not None:
                processed_labels.append(labels[idx])

    # Ritorna solo processed_texts se labels è None
    if labels is None:
        return processed_texts
    return processed_texts, processed_labels

In [8]:
# Preprocess dei testi di addestramento, validazione e test
processed_train_text, labels = process_text_batch(texts, labels)  # per il set di addestramento
processed_val_text, val_labels = process_text_batch(val_texts, val_labels)  # per il set di validazione
processed_test_text = process_text_batch(test_texts)  # per il set di test (senza etichette)


Processing texts: 100%|██████████| 784/784 [00:00<00:00, 4439.69it/s]


In [9]:
df_train = pd.DataFrame({
    'text': processed_train_text,
    'label': labels
})

In [10]:
df_train.head()

,text,label
0,seeing ppl walking w/ crutches makes me really...,1
1,"look for the girl with the broken smile, ask h...",0
2,Now I remember why I buy books online servicew...,1
3,So is he banded from wearing the clothes? karma,1
4,Just found out there are Etch A Sketch apps. o...,1


In [11]:
df_val = pd.DataFrame({
    'text': processed_val_text,
    'label': val_labels
})

In [12]:
df_test = pd.DataFrame({
    'text': processed_test_text,
})

In [13]:
processed_train_text

['seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life',
 'look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. :two_hearts::musical_note:',
 'Now I remember why I buy books online servicewithasmile',
 'So is he banded from wearing the clothes? karma',
 'Just found out there are Etch A Sketch apps. oldschool notoldschool',
 "Hey what do you know, one of the witnesses supporting Darren Wilson's story lied! And is racist! Mind blown!",
 'on stage at flzjingleball at the in tampa iheartradio',
 "You know it's going to be a great day when you're Garmin resets itself and you spill some cinnamon down yourself slowclap",
 'Halfway thorough my workday ... Woo <IRONY>',
 'Would like to thank my nephew for giving me his horrible cold & sore throat etc.. Much appreciated!',
 "I forked node! Get ready for the future. (Where's my interviews)",
 "I'm off to visit great-nephew very ill, only 20 fair",
 'If his Twitter ac

In [14]:
def augment_mask_text(tokens, mask_token="[MASK]", mask_prob=0.15):
    """
    Data augmentation che, con una probabilità mask_prob, sostituisce i token con mask_token.

    Args:
        tokens (list): Lista di token del tweet.
        mask_token (str): Il token da utilizzare per mascherare.
        mask_prob (float): Probabilità di mascherare ciascun token.

    Returns:
        list: Lista di token modificata.
    """
    augmented_tokens = []
    for token in tokens:
        if random.random() < mask_prob:
            augmented_tokens.append(mask_token)
        else:
            augmented_tokens.append(token)
    return augmented_tokens


In [15]:
def create_dataset(df, tokenizer = None):

    # Creiamo un dataset a partire dal dataframe
    dataset = Dataset.from_pandas(df)

    # Tokenizzazione del testo
    dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=128), batched=True)

    return dataset

In [16]:
# Funzione per creare un dataloader
def create_dataloader(dataset, batch_size, shuffle=False):

    # Combiniamo i dati
    def collate_fn(examples):
        return {
            'input_ids': torch.tensor([example['input_ids'] for example in examples]),
            'attention_mask': torch.tensor([example['attention_mask'] for example in examples]),
            'labels': torch.tensor([example['label'] for example in examples])
        }

    # Creiamo il dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=shuffle)

    # Ritorniamo il dataloader
    return dataloader

In [17]:
class IronyClassifier(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=128, dropout=0.3, model_name='microsoft/deberta-v3-base'):
        super(IronyClassifier, self).__init__()
        self.model = DebertaV2Model.from_pretrained(model_name)
        self.adaptation = nn.Linear(embedding_dim, embedding_dim)  # Layer trainabile sopra gli embedding
        self.norm = nn.LayerNorm(embedding_dim)
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.01),
            nn.Dropout(dropout),
            nn.LeakyReLU(0.01),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, input_ids, attention_mask):
        embeddings = self.model(input_ids, attention_mask).last_hidden_state[:, 0, :]
        embeddings = self.adaptation(embeddings)
        embeddings = self.norm(embeddings)
        return self.classifier(embeddings)

In [18]:
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    with tqdm(dataloader, desc='Training') as pbar:
        for batch in pbar:


            #print(f"Batch type: {type(batch)}, length: {len(batch)}")  # 👀 DEBUG
            #print(f"Batch content: {batch}")  # 👀 DEBUG


            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).float()

            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask)

            loss = nn.BCEWithLogitsLoss()(outputs.view(-1), labels)
            loss.backward()


            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += loss.item()

            # Raccolta per metriche
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend((torch.sigmoid(outputs).view(-1) > 0.4).long().cpu().numpy())

            # Calcola le metriche cumulative
            cumulative_accuracy = sum(1 for x, y in zip(all_preds, all_labels) if x == y) / len(all_labels)
            cumulative_f1 = f1_score(all_labels, all_preds, average='macro')

            # Aggiorna progress bar con loss
            pbar.set_postfix(accuracy=cumulative_accuracy, f1=cumulative_f1)


    # Calcolo metriche finali
    avg_loss = total_loss / len(dataloader)
    avg_accuracy = sum(1 for x, y in zip(all_preds, all_labels) if x == y) / len(all_labels)
    f1 = f1_score(all_labels, all_preds, average='macro')

    return avg_loss, avg_accuracy, f1


In [19]:
def evaluate(model, dataloader, device, test_mode=False):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = [] if not test_mode else None

    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Evaluation'):
            if test_mode:
               input_ids = batch['input_ids'].to(device)
               attention_mask = batch['attention_mask'].to(device)
            else:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device).float()

            outputs = model(input_ids, attention_mask)

            pred_labels = (torch.sigmoid(outputs) > 0.4).long().cpu().numpy()
            predictions.extend(pred_labels)

            if not test_mode:
                loss = nn.BCEWithLogitsLoss()(outputs.view(-1), labels.view(-1))
                total_loss += loss.item()
                true_labels.extend(labels.cpu().numpy())

    if test_mode:
        return predictions
    else:
        avg_loss = total_loss / len(dataloader)
        accuracy = sum(1 for x, y in zip(predictions, true_labels) if x == y) / len(true_labels)
        f1 = f1_score(true_labels, predictions, average='macro')
        return avg_loss, accuracy, f1


In [20]:
# Initialize feature extractor and model
#feature_extractor = TweetFeatureExtractor()
model = IronyClassifier().to(gpu_device)

In [21]:
batch_size = 16

# Create datasets
train_dataset = create_dataset(df_train, tokenizer)

val_dataset = create_dataset(df_val, tokenizer)

test_dataset = create_dataset(df_test, tokenizer)

# Create dataloaders
train_loader = create_dataloader(train_dataset, batch_size, shuffle = True)
val_loader = create_dataloader(val_dataset, batch_size)
test_loader = create_dataloader(test_dataset, batch_size)


Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

Map:   0%|          | 0/954 [00:00<?, ? examples/s]

Map:   0%|          | 0/784 [00:00<?, ? examples/s]

In [22]:
train_dataset[0]

{'text': 'seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life',
 'label': 1,
 'input_ids': [1,
  1769,
  44586,
  2220,
  2976,
  320,
  46674,
  682,
  351,
  431,
  2199,
  270,
  262,
  498,
  404,
  1033,
  265,
  312,
  432,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [23]:
for batch in train_loader:
    print(batch)
    break

{'input_ids': tensor([[    1,   263,   339,  ...,     0,     0,     0],
        [    1,  3411,   523,  ...,     0,     0,     0],
        [    1,   273,  2218,  ...,     0,     0,     0],
        ...,
        [    1,  7128,   299,  ...,     0,     0,     0],
        [    1, 46077,   272,  ...,     0,     0,     0],
        [    1,  2392,   280,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0])}


In [24]:
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0):
        """
        Inizializza l'oggetto EarlyStopping.
        :param patience: Numero massimo di epoche senza miglioramento.
        :param min_delta: Soglia minima per considerare un miglioramento.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False
        self.best_metric = -torch.inf  # Per val_f1, inizializzare a -inf

    def __call__(self, current_metric):
        """
        Controlla se la metrica (ad es. val_f1) non migliora oltre la soglia (min_delta).
        """
        if (current_metric <= self.best_metric):  # Nessun miglioramento
            self.counter += 1
            x_counter = "❌" * self.counter
            print(f'{x_counter} {self.counter} epoca senza miglioramenti')
            if self.counter == self.patience:
                self.early_stop = True
                print("Early stopping Attivato!")
        else:  # Miglioramento della metrica
            self.best_metric = current_metric
            self.counter = 0

In [25]:
import traceback

def train_model(train_loader, val_loader, num_epochs=5, device=None):
    device = device or torch.device(gpu_device)
    print(f"Using device: {device}")

    model = IronyClassifier().to(device)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=2e-5,
        betas=(0.9, 0.999),
        eps=1e-8,
        weight_decay=0.03
    )

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.1,
        patience=3,
        verbose=True
    )

    # Inizializza EarlyStopping con val_f1 come metrica
    early_stopping = EarlyStopping(patience=4, min_delta=0.002)
    best_val_f1 = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        try:
            # Train and validate
            train_loss, train_acc, train_f1 = train_epoch(model, train_loader, optimizer, device)
            val_loss, val_acc, val_f1 = evaluate(model, val_loader, device)

            print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}, Training F1: {train_f1:.4f}")
            print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}, Validation F1: {val_f1:.4f}")

            # Update scheduler
            scheduler.step(val_loss)

            # Check for best model and save
            if val_f1 > best_val_f1:
                best_val_f1 = val_f1
                torch.save(model.state_dict(), 'best_irony_model.pth')
                print("📌 Saved new best model!")

            # Early stopping check
            early_stopping(val_f1)
            if early_stopping.early_stop:
                print("⏹ Early stopping triggered.")
                break

        except Exception as e:
            print(f"⚠ Error during epoch {epoch + 1}: {str(e)}")
            print(traceback.format_exc())  # Stampa il traceback completo per il debugging
            break

    print("\n✅ Training completed!")
    return model


In [26]:
device = torch.device(gpu_device)
model = train_model(train_loader, val_loader, num_epochs=12)

Using device: cuda

Epoch 1/12


Evaluation: 100%|██████████| 60/60 [00:08<00:00,  7.06it/s]


Training Loss: 0.6793, Training Accuracy: 0.5821, Training F1: 0.5724
Validation Loss: 0.5705, Validation Accuracy: 0.6939, Validation F1: 0.6939
📌 Saved new best model!

Epoch 2/12


Evaluation: 100%|██████████| 60/60 [00:08<00:00,  7.02it/s]


Training Loss: 0.5495, Training Accuracy: 0.7282, Training F1: 0.7280
Validation Loss: 0.5862, Validation Accuracy: 0.6761, Validation F1: 0.6673
❌ 1 epoca senza miglioramenti

Epoch 3/12


...

Training Loss: 0.0753, Training Accuracy: 0.9976, Training F1: 0.9976
Validation Loss: 0.8364, Validation Accuracy: 0.7233, Validation F1: 0.7233
❌ 1 epoca senza miglioramenti

Epoch 11/12


Evaluation: 100%|██████████| 60/60 [00:08<00:00,  7.06it/s]


Training Loss: 0.0791, Training Accuracy: 0.9965, Training F1: 0.9965
Validation Loss: 0.8493, Validation Accuracy: 0.7180, Validation F1: 0.7180
❌❌ 2 epoca senza miglioramenti

Epoch 12/12


Evaluation: 100%|██████████| 60/60 [00:08<00:00,  7.03it/s]

Training Loss: 0.0792, Training Accuracy: 0.9958, Training F1: 0.9958
Validation Loss: 0.8426, Validation Accuracy: 0.7149, Validation F1: 0.7147
❌❌❌ 3 epoca senza miglioramenti

✅ Training completed!
